In [1]:
import os
import pandas as pd
import numpy as np
import csv
import json
import math

try: 
    __file__
except:
    curr_dir = os.path.abspath('')
else:
    curr_dir = os.path.dirname(os.path.abspath(__file__))
    
app_root = curr_dir if os.path.basename(curr_dir) != "src" else os.path.dirname(curr_dir)

destdir = os.path.join(app_root,'data','manipulated_data')      # non-docker stay in repository


file = os.path.join(destdir,'Exogenous_better_excel.xlsx')
almostcomplete_data = pd.read_excel(file, 'Sheet1', index_col=None)

almostcomplete_data

,Target Location,Exogenous Location,Mean Absolute Error,Better model
0,"Albemarle, NC",NaN,1.590524,NaN
1,NaN,"CHESTERFIELD 3 E, SC', ' CHERAW, SC', 'PAGELAN...",2.083258,0.0
2,NaN,"'CHESTERFIELD 3 E, SC'",1.942021,0.0
3,NaN,"' CHERAW, SC'",1.830673,0.0
4,NaN,"'PAGELAND 9.0 WNW, SC'",2.027484,0.0
5,NaN,"'FORT MILL 4 NW, SC'",1.788067,0.0
6,NaN,"'CHESTERFIELD 3 E, SC', ' CHERAW, SC'",1.978665,0.0
7,NaN,"CHESTERFIELD 3 E, SC', 'PAGELAND 9.0 WNW, SC'",2.032507,0.0
8,NaN,"CHESTERFIELD 3 E, SC', 'FORT MILL 4 NW, SC'",1.981879,0.0
9,NaN,"CHERAW, SC','PAGELAND 9.0 WNW, SC'",2.040195,0.0


In [2]:
# Data Massage
# remove exmae text from Target Location column to match predetermined format
import re

almostcomplete_data = almostcomplete_data.drop(['Better model'], axis=1)

unwanted_targetloc_regex = r"^[ ]*exmae[ =]*$"
keymae_regex = r"^keymae of: (.*)$"
exogconv1_regex = r"^[' ]*(.*?)[ ]*[',]*$"
exogconv2_regex = r"',[ ]*'[ ]*"

for key in almostcomplete_data.index:
    value = almostcomplete_data['Target Location'][key]
    if isinstance(value, str) and re.match(unwanted_targetloc_regex, value) is not None:
        almostcomplete_data.at[key, 'Target Location'] = math.nan
    elif isinstance(value, str) and re.match(keymae_regex, value) is not None:
        almostcomplete_data.at[key, 'Target Location'] = re.sub(keymae_regex, r'\1', value).upper().strip(' ')
    elif isinstance(value, str):
        almostcomplete_data.at[key, 'Target Location'] = value.upper().strip(' ')
    
    exogloc_value = almostcomplete_data['Exogenous Location'][key]
    if isinstance(exogloc_value, str):
        exogloc_value2 = re.sub(exogconv1_regex, r'\1', exogloc_value)
        exogloc_value3 = "|".join(re.split(exogconv2_regex, exogloc_value2))
        almostcomplete_data.at[key, 'Exogenous Location'] = exogloc_value3.upper()


# UPPERCASE all Target Locations, and Exogenous Location names
almostcomplete_data.tail(15)

,Target Location,Exogenous Location,Mean Absolute Error
397,NaN,"FLORENCE REGIONAL AIRPORT, SC|CHESTERFIELD 3 E...",2.362032
398,NaN,"FLORENCE 8 NE, SC|CHESTERFIELD 3 E, SC|CHERAW, SC",2.304465
399,NaN,"DARLINGTON, SC|DILLON, SC|FLORENCE REGIONAL AI...",2.134537
400,NaN,"DARLINGTON, SC|DILLON, SC|FLORENCE REGIONAL AI...",2.077845
401,NaN,"DARLINGTON, SC|DILLON, SC|FLORENCE REGIONAL AI...",2.152721
402,NaN,"DARLINGTON, SC|DILLON, SC|FLORENCE 8 NE, SC|CH...",2.138086
403,NaN,"DARLINGTON, SC|DILLON, SC|FLORENCE 8 NE, SC|CH...",2.131730
404,"BOONE 1 SE, NC",NaN,NaN
405,"DANBURY, NC",NaN,NaN
406,"EDEN, NC",NaN,NaN


In [3]:

almostcomplete_dict = {}

for i in almostcomplete_data.index:
    value = almostcomplete_data['Target Location'][i]
    if isinstance(value, str) and not math.isnan(almostcomplete_data['Mean Absolute Error'][i]):
        almostcomplete_dict[value] = { 'keymae': almostcomplete_data['Mean Absolute Error'][i], 'exogen': {} }
        
    elif isinstance(value, str) and math.isnan(almostcomplete_data['Mean Absolute Error'][i]):
        almostcomplete_data.drop(i,axis=0)  # remove unknown keymaes

print(json.dumps(almostcomplete_dict, indent=4))

{
    "ALBEMARLE, NC": {
        "keymae": 1.59052394486363,
        "exogen": {}
    },
    "ARCOLA, NC": {
        "keymae": 1.54425586234438,
        "exogen": {}
    },
    "BURLINGTON, NC": {
        "keymae": 0.788801134951249,
        "exogen": {}
    },
    "ENFIELD, NC": {
        "keymae": 0.912938951758521,
        "exogen": {}
    },
    "GREENSBORO AP, NC": {
        "keymae": 1.62055111053581,
        "exogen": {}
    },
    "HENDERSON 2 NNW, NC": {
        "keymae": 1.60022457843925,
        "exogen": {}
    },
    "JACKSON SPRINGS 5 WNW, NC": {
        "keymae": 1.88013588075888,
        "exogen": {}
    },
    "LAURINBURG, NC": {
        "keymae": 1.64842225744649,
        "exogen": {}
    },
    "ROANOKE RAPIDS, NC": {
        "keymae": 1.97334050874123,
        "exogen": {}
    },
    "MURFREESBORO, NC": {
        "keymae": 1.71910997032104,
        "exogen": {}
    },
    "LUMBERTON AREA, NC": {
        "keymae": 1.69981932215264,
        "exogen": {}
    },
    "LO

In [4]:
# Collect exmae values and insert into dictionary nested under the associated keymae

dict_keys = list(almostcomplete_dict.keys())
current_dict_index = -1
collecting = False

i = 0
while i < len(almostcomplete_data.index):
    exogloc = almostcomplete_data['Exogenous Location'][i]
    if isinstance(exogloc, str) and collecting:
#         print("collecting... almostcomplete_dict[dict_keys[{0}]]['exogen'][{1}]".format(current_dict_index, exogloc))
        key = dict_keys[current_dict_index]
        almostcomplete_dict[key]['exogen'][exogloc] = { 'exmae': almostcomplete_data['Mean Absolute Error'][i] }
    elif isinstance(exogloc, str) and not collecting:
        found_keymae = almostcomplete_data['Target Location'][i-1]
        if found_keymae in dict_keys:
            current_dict_index = dict_keys.index(found_keymae)
            collecting = True
            i -= 1
        else:
            print("ERROR: Found exogloc ({0}) does not exist in almostcomplete_dict".format(exogloc))
    else:
        collecting = False
        
    i += 1

print(json.dumps(almostcomplete_dict, indent=4))
# with open(os.path.join(destdir, "almostThere.json"), "w") as almost_there_file:
#     almost_there_file.write( json.dumps(almostcomplete_dict, indent=4) )


{
    "ALBEMARLE, NC": {
        "keymae": 1.59052394486363,
        "exogen": {
            "CHESTERFIELD 3 E, SC|CHERAW, SC|PAGELAND 9.0 WNW, SC|FORT MILL 4 NW, SC": {
                "exmae": 2.08325758240936
            },
            "CHESTERFIELD 3 E, SC": {
                "exmae": 1.94202068068645
            },
            "CHERAW, SC": {
                "exmae": 1.83067341056705
            },
            "PAGELAND 9.0 WNW, SC": {
                "exmae": 2.0274841948442
            },
            "FORT MILL 4 NW, SC": {
                "exmae": 1.78806744315493
            },
            "CHESTERFIELD 3 E, SC|CHERAW, SC": {
                "exmae": 1.97866499847341
            },
            "CHESTERFIELD 3 E, SC|PAGELAND 9.0 WNW, SC": {
                "exmae": 2.03250744234255
            },
            "CHESTERFIELD 3 E, SC|FORT MILL 4 NW, SC": {
                "exmae": 1.98187923521911
            },
            "CHERAW, SC|PAGELAND 9.0 WNW, SC": {
                "exma

Now that all the data is pulled out of a DataFrame and now in an equivalent Dictionary object.  Merge with AllMAE.json file

In [4]:
all_results_filename = os.path.join(destdir, 'allMAE.json')

In [5]:

all_results_filename = os.path.join(destdir, 'allMAE.json')
with open(all_results_filename, "r") as all_results_file:
    all_results = json.loads(all_results_file.read())

target_location_list = list(almostcomplete_dict.keys())

for i in range(0, len(target_location_list)):
    key = target_location_list[i]
    if key not in all_results:
        raise TypeError("Target Location Not Found ({0})".format(key))
    
    if all_results[key]['keymae'] is not None:
        if round(all_results[key]['keymae'],9) != round(almostcomplete_dict[key]['keymae'],9):
            print("MISMATCH ERROR: (all_results[{0}]['keymae'] = {1}) != {2}".format(key, all_results[key]['keymae'], almostcomplete_dict[key]['keymae']))
    else:
        all_results[key]['keymae'] = round(almostcomplete_dict[key]['keymae'],9)
    
    exog_list = list(almostcomplete_dict[key]['exogen'].keys())
    for j in range(0, len(exog_list)):
        exog_key = exog_list[j]
        if exog_key not in all_results[key]['exogen']:
            print("ExogLocation in {0} Not Found ({1})".format(key, exog_key))
            continue
        
        if all_results[key]['exogen'][exog_key]['exmae'] is not None:
            if round(all_results[key]['exogen'][exog_key]['exmae'], 9) != round(almostcomplete_dict[key]['exogen'][exog_key]['exmae'], 9):
                print("MISMATCH ERROR: (all_results[{0}]['exogen'][{1}]['exmae'] = {2}) != {3}".format(
                    key,
                    exog_key,
                    all_results[key]['exogen'][exog_key]['exmae'],
                    almostcomplete_dict[key]['exogen'][exog_key]['exmae']
                ))
            else:  # simplify all answers to 9 digits
                all_results[key]['exogen'][exog_key]['exmae'] = round(all_results[key]['exogen'][exog_key]['exmae'],9)
        else:
            all_results[key]['exogen'][exog_key]['exmae'] = round(almostcomplete_dict[key]['exogen'][exog_key]['exmae'], 9)
        

# print(json.dumps(all_results, indent=4, sort_keys=True))
# with open(all_results_filename, "w") as all_results_file:
#     all_results_file.write( json.dumps(all_results, indent=4, sort_keys=True)+'\n' )
#     print("Exported to File: {0}".format(all_results_filename))

MISMATCH ERROR: (all_results[ARCOLA, NC]['keymae'] = 1.5479680814163455) != 1.54425586234438
MISMATCH ERROR: (all_results[GRANDFATHER MTN, NC]['keymae'] = 2.583667713) != 2.58806403787115
MISMATCH ERROR: (all_results[GRANDFATHER MTN, NC]['exogen'][ELIZABETHTON, TN|ROAN MOUNTAIN 3SW, TN]['exmae'] = 2.797569592) != 2.97106152453339
MISMATCH ERROR: (all_results[GRANDFATHER MTN, NC]['exogen'][ELIZABETHTON, TN]['exmae'] = 2.544987837) != 2.69524767468778
MISMATCH ERROR: (all_results[GRANDFATHER MTN, NC]['exogen'][ROAN MOUNTAIN 3SW, TN]['exmae'] = 2.799639392) != 2.97845074957975
Exported to File: /Users/gbpackers225/Development/lilfetz22/CapstoneProject1/data/manipulated_data/allMAE.json


### Now remove leftover placeholders that still exist in preparation for recalculation

In [9]:

with open(all_results_filename, "r") as all_results_file:
    all_results = json.loads(all_results_file.read())

targetlocs = list(all_results.keys())
for i in range(0, len(targetlocs)):
    key = targetlocs[i]
    
    if all_results[key]['keymae'] is None:
        all_results[key].pop('data_source_sha1')
        print("MISSING KEYMAE: all_results[{0}]['keymae'] = {1}".format(key, all_results[key]['keymae']))
        
    elif key in almostcomplete_dict:
        if round(all_results[key]['keymae'],9) != round(almostcomplete_dict[key]['keymae'],9):
            print("SOLVING MISMATCH ERROR (reset): all_results[{0}]['keymae'] = None".format(key))
            all_results[key]['keymae'] = None
            all_results[key].pop('data_source_sha1')
    
    exog_list = list(all_results[key]['exogen'].keys())
    for j in range(0, len(exog_list)):
        exog_key = exog_list[j]
        
        if all_results[key]['exogen'][exog_key]['exmae'] is None:
            all_results[key]['exogen'].pop(exog_key)
            print("MISSING EXMAE: all_results[{0}]['exogen'][{1}]".format(key, exog_key))
            
        elif key in almostcomplete_dict and exog_key in almostcomplete_dict[key]['exogen']:
            if round(all_results[key]['exogen'][exog_key]['exmae'], 9) != round(almostcomplete_dict[key]['exogen'][exog_key]['exmae'], 9):
                print("SOLVING MISMATCH ERROR (reset): all_results[{0}]['exogen'][{1}]['exmae'] = None".format(key, exog_key))
                all_results[key]['exogen'][exog_key]['exmae'] = None
                all_results[key]['exogen'][exog_key].pop('data_source_sha1')
            
# print(json.dumps(all_results, indent=4, sort_keys=True))
with open(all_results_filename, "w") as all_results_file:
    all_results_file.write( json.dumps(all_results, indent=4, sort_keys=True)+'\n' )
    print("Exported to File: {0}".format(all_results_filename))
    

SOLVING MISMATCH ERROR (reset): all_results[ARCOLA, NC]['keymae'] = None
MISSING EXMAE: all_results[BURLINGTON, NC]['exogen'][CHATHAM, VA]
MISSING EXMAE: all_results[BURLINGTON, NC]['exogen'][CHATHAM, VA|MARTINSVILLE FILTER PLANT, VA]
MISSING EXMAE: all_results[BURLINGTON, NC]['exogen'][CHATHAM, VA|MARTINSVILLE FILTER PLANT, VA|SOUTH BOSTON, VA]
MISSING EXMAE: all_results[BURLINGTON, NC]['exogen'][CHATHAM, VA|SOUTH BOSTON, VA]
MISSING EXMAE: all_results[BURLINGTON, NC]['exogen'][MARTINSVILLE FILTER PLANT, VA|SOUTH BOSTON, VA]
MISSING EXMAE: all_results[BURLINGTON, NC]['exogen'][SOUTH BOSTON, VA]
SOLVING MISMATCH ERROR (reset): all_results[GRANDFATHER MTN, NC]['keymae'] = None
SOLVING MISMATCH ERROR (reset): all_results[GRANDFATHER MTN, NC]['exogen'][ELIZABETHTON, TN]['exmae'] = None
SOLVING MISMATCH ERROR (reset): all_results[GRANDFATHER MTN, NC]['exogen'][ELIZABETHTON, TN|ROAN MOUNTAIN 3SW, TN]['exmae'] = None
SOLVING MISMATCH ERROR (reset): all_results[GRANDFATHER MTN, NC]['exogen'][

In [8]:
# adjusting values down to 9 digits

with open(all_results_filename, "r") as all_results_file:
    all_results = json.loads(all_results_file.read())

targetlocs = list(all_results.keys())
for i in range(0, len(targetlocs)):
    key = targetlocs[i]
    
    if all_results[key]['keymae'] is not None:
        current_keymae = all_results[key]['keymae']
        if round(current_keymae, 9) != current_keymae:             # it is not rounded
            all_results[key]['keymae'] = round(current_keymae, 9)
            print("Adjusted keymae value: (orig[{0}] = {1}) => {2}".format(key, current_keymae, all_results[key]['keymae']))
            
    if 'exogen' in all_results[key]:
        exog_list = list(all_results[key]['exogen'].keys())
        for j in range(0, len(exog_list)):
            exog_key = exog_list[j]
        
            if all_results[key]['exogen'][exog_key]['exmae'] is not None:
                current_exmae = all_results[key]['exogen'][exog_key]['exmae']
                if round(current_exmae, 9) != current_exmae:             # it is not rounded
                    all_results[key]['exogen'][exog_key]['exmae'] = round(current_exmae, 9)
                    print("Adjusted exmae value: (orig[{0}][{1}] = {2}) => {3}".format(key, exog_key, current_exmae, all_results[key]['exogen'][exog_key]['exmae']))
            
# print(json.dumps(all_results, indent=4, sort_keys=True))
# with open(all_results_filename, "w") as all_results_file:
#     all_results_file.write( json.dumps(all_results, indent=4, sort_keys=True)+'\n' )
#     print("Exported to File: {0}".format(all_results_filename))
    

Adjusted keymae value: (orig[BOONE 1 SE, NC] = 2.4809657084670653) => 2.480965708
Adjusted exmae value: (orig[BOONE 1 SE, NC][ROAN MOUNTAIN 3SW, TN] = 3.1150406350016864) => 3.115040635
Adjusted keymae value: (orig[CASAR, NC] = 1.9931330930000002) => 1.993133093
Adjusted keymae value: (orig[DANBURY, NC] = 1.5765827847357705) => 1.576582785
Adjusted exmae value: (orig[DANBURY, NC][MARTINSVILLE FILTER PLANT, VA] = 1.8526283068064833) => 1.852628307
Adjusted exmae value: (orig[DANBURY, NC][MARTINSVILLE FILTER PLANT, VA|MEADOWS OF DAN 5 SW, VA] = 2.082558684944888) => 2.082558685
Adjusted exmae value: (orig[DANBURY, NC][MARTINSVILLE FILTER PLANT, VA|MEADOWS OF DAN 5 SW, VA|STUART, VA] = 2.021224928021283) => 2.021224928
Adjusted exmae value: (orig[DANBURY, NC][MARTINSVILLE FILTER PLANT, VA|STUART, VA] = 2.043754322484901) => 2.043754322
Adjusted exmae value: (orig[DANBURY, NC][MEADOWS OF DAN 5 SW, VA] = 1.913366953756763) => 1.913366954
Adjusted exmae value: (orig[DANBURY, NC][MEADOWS OF D